# DATA TRANSFORMATION - 19_consolidacion
## 0. Previos

Se cargan las bases y las librerías a utilizar.

In [1]:
# Librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np

import generic_funcions as gf
import adherence_functions as af
import warnings
warnings.filterwarnings('ignore')

ruta_archivos = 'C:/Users/monic/documentos/ds4a/project/datos/'

## 1. Varios

In [2]:
calculos = gf.diccionario_agg_functions()
dcc = gf.diccionario_llaves()

In [3]:
# execute initial file
files = 'C:/Users/monic/documentos/ds4a/ds4a_team80/1_data_transformation/1_consolidacion_mensual/'
file_0 = files + '0_initial.py'

exec(open(file_0).read())

## 2. Cálculo de Variables por Módulo

### 2.1. Funciones

In [4]:
# Para el cálculo de variables 
def cal_variables( base_features, base_ad_12, idx, nmeses, calculos, cc, dc_n, base_ad_features  ):
    
    # Cruza con adherencia
    base_cruce = base_ad_12.merge( base_features , how = 'inner', 
                            left_on=['id','year_c','month_c'], right_on = ['id','year','month'] )
    
    # Limita el número de meses
    base_1 = base_cruce[base_cruce['d_meses'] <= nmeses]
    
    # Realiza agrupaciones de acuerdo al cálculo
    for calculo in calculos[ cc ]:
        
        if calculo == 'sum' :
            variables = calculos[ cc ][calculo]
            #print(variables)
            #print(nmeses)
            base_12 = base_1.groupby( idx )[variables].sum().reset_index()
            
            sufi = '_' + calculo + '_' + str(nmeses)
            base_12.columns = [ s + sufi for s in base_12.columns]
            base_12.rename( columns={ 'id' + sufi :'id','year_obs' + sufi :'year_obs', 'month_obs' + sufi :'month_obs' }, inplace=True)
            dc_n[cc][ calculo + '_' + str(nmeses) ] = base_12
            
            base_ad_features = base_ad_features.merge( base_12 , how = 'left', 
                       left_on = ['id','year','month'], right_on = ['id','year_obs','month_obs'] )
            
            base_ad_features = base_ad_features.drop(['year_obs', 'month_obs'], axis=1)
            
            
        elif calculo == 'avg' :
            variables = calculos[ cc ][calculo]
            #print(variables)
            #print(nmeses)
            base_12 = base_1.groupby( idx )[variables].mean().reset_index()
            
            sufi = '_' + calculo + '_' + str(nmeses)
            base_12.columns = [ s + sufi for s in base_12.columns]
            base_12.rename( columns={ 'id' + sufi :'id','year_obs' + sufi :'year_obs', 'month_obs' + sufi :'month_obs' }, inplace=True)
            dc_n[cc][ calculo + '_' + str(nmeses) ] = base_12
            
            base_ad_features = base_ad_features.merge( base_12 , how = 'left', 
                       left_on = ['id','year','month'], right_on = ['id','year_obs','month_obs'] )
            
            base_ad_features = base_ad_features.drop(['year_obs', 'month_obs'], axis=1)
        
            
    return dc_n, base_ad_features


In [7]:
# Para el cálculo de variables 

def cal_variables_f( base_features, base_ad_12, idx, calculos, cc, dc_n, base_ad_features  ):
    
    # Cruza con adherencia
    base_cruce = base_ad_12.merge( base_features , how = 'inner', 
                            left_on=['id','year_c','month_c'], right_on = ['id','year','month'] )
    
    # Realiza agrupaciones de acuerdo al cálculo
    for calculo in calculos[ cc ]:        
        
        if calculo == 'foto' :
            
            # Limita el número de meses
            base_1 = base_cruce[base_cruce['d_meses'] == 1]
    
            variables = calculos[ cc ][calculo] + idx

            base_12 = base_1[ variables]
            
            sufi = '_' + calculo 
            base_12.columns = [ s + sufi for s in base_12.columns]
            base_12.rename( columns={ 'id' + sufi :'id','year_obs' + sufi :'year_obs', 'month_obs' + sufi :'month_obs' }, inplace=True)
            dc_n[cc][ calculo ] = base_12
            
            base_ad_features = base_ad_features.merge( base_12 , how = 'left', 
                       left_on = ['id','year','month'], right_on = ['id','year_obs','month_obs'] )
            
            base_ad_features = base_ad_features.drop(['year_obs', 'month_obs'], axis=1)
            
        elif calculo == 'flag' :
            
            base_1 = base_cruce
            variables = calculos[ cc ][calculo]
            
            base_12 = base_1.groupby( idx )[variables].max().reset_index()
            
            #for vv in variables :
            #    print(vv)
            #    base_12[vv] = base_12[vv].apply( lambda x: 1 if x >= 1 else 0 )
            
            sufi = '_' + calculo 
            base_12.columns = [ s + sufi for s in base_12.columns]
            base_12.rename( columns={ 'id' + sufi :'id','year_obs' + sufi :'year_obs', 'month_obs' + sufi :'month_obs' }, inplace=True)
            dc_n[cc][ calculo ] = base_12
            
            base_ad_features = base_ad_features.merge( base_12 , how = 'left', 
                       left_on = ['id','year','month'], right_on = ['id','year_obs','month_obs'] )
            
            base_ad_features = base_ad_features.drop(['year_obs', 'month_obs'], axis=1)
        
    return dc_n, base_ad_features


### 2.2. Varios

In [5]:
base_ini = gf.base_ids_mensual( ruta_archivos )
base_ad_12, base_ad = af.base_features_adeherencia0( ruta_archivos, dcc, ids )

In [10]:
base_ad_12[ ( base_ad_12['id']==500547) & ( base_ad_12['year_obs']==2019) & ( base_ad_12['month_obs']==3) ].head(5)

,id,year_obs,month_obs,adeherencia_0,key,year_c,month_c,d_meses
60,500547,2019,3,1,1,2016,1,38
61,500547,2019,3,1,1,2016,2,37
62,500547,2019,3,1,1,2016,3,36
63,500547,2019,3,1,1,2016,4,35
64,500547,2019,3,1,1,2016,5,34


In [11]:
base_ad[ ( base_ad['id']==500547) & ( base_ad['year']==2019) & ( base_ad['month']==3) ]

,id,year,month,adeherencia_0,key
1,500547,2019,3,1,1


In [10]:
print(base_ini.shape)
base_ini.head()

(52320, 3)


,id,year,month
0,500547,2016,1
1,500547,2016,2
2,500547,2016,3
3,500547,2016,4
4,500547,2016,5


In [8]:
idx = ['id', 'year_obs', 'month_obs']
nmeses = [6,12]
dc_n = {}

base_ad_features = base_ad

for cc in calculos :
    
    # Ejecuta el modulo
    m = calculos[cc]['mod']
    file_i = files + dcc[ m ]['prefi'] + '.py'
    exec(open(file_i).read())
    exec("base_i = base_final_" + dcc[m]['prefi'] )
    print( "Ejecutando modulo " + cc )
    print( base_i.shape )
    
    # Quita los nulls
    cols_i = base_i.columns

    base_i['nuls'] = base_i.isnull().sum(axis = 1)
    todo_null = base_i.shape[1] - 4    #id, month, year, nuls
    base_features = base_i[base_i['nuls'] < todo_null ]
    base_features.reset_index( inplace = True )
    base_features = base_features[cols_i]
    print(base_features.shape)
    
    # Calcula las variables
    dc_n[ cc ] = {}
    
    # Calculos sum, avg
    for mn in nmeses : 
        dc_n, base_ad_features = cal_variables( base_features, base_ad_12, idx, mn, calculos, cc, dc_n, base_ad_features  )
        
    # Calculos foto
    dc_n, base_ad_features = cal_variables_f( base_features, base_ad_12, idx, calculos, cc, dc_n, base_ad_features )


Ejecutando modulo far
(52320, 14)
(248, 14)
Ejecutando modulo hos
(52320, 11)
(576, 11)
Ejecutando modulo med
(52320, 9)
(19162, 9)
Ejecutando modulo urg
(52320, 7)
(2059, 7)
Ejecutando modulo vac
(52320, 4)
(215, 4)
Ejecutando modulo cal
(52320, 7)
(227, 7)
Ejecutando modulo act
(52320, 4)
(1217, 4)
Ejecutando modulo acd
(52320, 8)
(1211, 8)
Ejecutando modulo ant
(52320, 5)
(903, 5)
Ejecutando modulo anf
(52320, 5)
(9, 5)
Ejecutando modulo ayu
(52320, 4)
(2252, 4)
Ejecutando modulo bio
(52320, 9)
(8800, 9)
Ejecutando modulo epo
(52320, 9)
(124, 9)
Ejecutando modulo hab
(52320, 6)
(103, 6)
Ejecutando modulo inc
(52320, 5)
(4987, 5)
Ejecutando modulo imc
(53867, 4)
(8122, 4)


In [9]:
print(base_ad_features.shape)

cols_i = base_ad_features.columns

base_ad_features['nuls'] = base_ad_features.isnull().sum(axis = 1)
todo_null = base_ad_features.shape[1] - 6    #id, month, year, adh, key, nuls
base_ad_features = base_ad_features[base_ad_features['nuls'] < todo_null ]
base_ad_features.reset_index( inplace = True )
base_ad_features = base_ad_features[cols_i]
base_ad_features.shape

(5085, 139)


(3663, 151)

In [12]:
base_ad_features.head()

,id,year,month,adeherencia_0,key,far_rea_asma_sum_6,far_rea_total_sum_6,far_evo_aun_sum_6,far_cau_def_sum_6,far_cau_otra_sum_6,...,hab_alcohol_avg_12,hab_cigarrillo_avg_12,hab_ejercicio_avg_12,inc_con_sum_6,inc_inc_sum_6,inc_con_sum_12,inc_inc_sum_12,imc_avg_6,imc_avg_12,imc_foto
0,500547,2016,7,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,3.0,0.0,3.0,NaN,NaN,NaN
1,500547,2019,3,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,500547,2019,10,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.8,NaN
3,500547,2019,12,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.8,NaN
4,500547,2020,1,1,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.8,NaN


In [13]:
base_ad_features.groupby(['year','adeherencia_0'])['id'].count()

year  adeherencia_0
2016  0                 98
      1                329
2017  0                244
      1                654
2018  0                293
      1                709
2019  0                207
      1                781
2020  0                 30
      1                318
Name: id, dtype: int64